In [1]:
import numpy as np
from collections import Counter

def gini(y):
    """Gini Impurity"""
    hist = np.bincount(y)
    probs = hist / len(y)
    return 1 - np.sum(probs ** 2)

def split(X_column, threshold):
    """Split function for thresholding"""
    left = lambda X: X < threshold
    right = lambda X: X >= threshold
    return left, right

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # For leaf nodes

    def is_leaf(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _best_split(self, X, y):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] < threshold
                right_mask = ~left_mask
                if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
                    continue

                p = float(len(y[left_mask])) / len(y)
                gain = gini(y) - (p * gini(y[left_mask]) + (1 - p) * gini(y[right_mask]))

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature
                    split_thresh = threshold

        return split_idx, split_thresh

    def _build_tree(self, X, y, depth=0):
        num_samples_per_class = Counter(y)
        most_common_class = num_samples_per_class.most_common(1)[0][0]

        if depth >= self.max_depth or len(set(y)) == 1:
            return Node(value=most_common_class)

        feature, threshold = self._best_split(X, y)
        if feature is None:
            return Node(value=most_common_class)

        left_idx = X[:, feature] < threshold
        right_idx = ~left_idx

        left = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right = self._build_tree(X[right_idx], y[right_idx], depth + 1)
        return Node(feature, threshold, left, right)

    def _predict(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred))


Decision Tree Accuracy: 0.9666666666666667
